<a href="https://colab.research.google.com/github/christee8/lsa-model/blob/master/models/electra_cs/electra_cs_from_tf_to_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
!pip3 install --upgrade transformers

     |████████████████████████████████| 778kB 2.6MB/s 
     |████████████████████████████████| 3.0MB 13.5MB/s 
     |████████████████████████████████| 890kB 26.7MB/s 
     |████████████████████████████████| 1.1MB 43.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=5161cce45b4ddc8035b4c273c60c2e7e2b6c47083374c9d03b535aa5887f7a28
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
!cp -r /gdrive/'My Drive'/question_answering/models/electra_cs/model_base .

In [4]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 37184 (delta 7), reused 10 (delta 0), pack-reused 37160
Receiving objects: 100% (37184/37184), 27.11 MiB | 11.41 MiB/s, done.
Resolving deltas: 100% (25720/25720), done.


In [5]:
%env ELECTRA_BASE_DIR=/content/model_base

env: ELECTRA_BASE_DIR=/content/model_base


In [6]:
!ls $ELECTRA_BASE_DIR

checkpoint_files			   electra_cs_base_vocab.txt
electra_cs_base_discriminator.bin	   graph.pbtxt
electra_cs_base_discriminator_config.json


In [7]:
# https://github.com/huggingface/transformers/blob/master/src/transformers/convert_electra_original_tf_checkpoint_to_pytorch.py

!python3 ./transformers/src/transformers/convert_electra_original_tf_checkpoint_to_pytorch.py \
--tf_checkpoint_path $ELECTRA_BASE_DIR/checkpoint_files/model.ckpt-847000 \
--pytorch_dump_path $ELECTRA_BASE_DIR/electra_cs_base_discriminator.bin \
--discriminator_or_generator 'discriminator' \
--config_file $ELECTRA_BASE_DIR/electra_cs_base_discriminator_config.json

2020-08-10 08:56:37.802190: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Building PyTorch model from configuration: ElectraConfig {
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "type_vocab_size": 2,
  "vocab_size": 127847
}

INFO:transformers.modeling_electra:Converting TensorFlow checkpoint from /content/model_base/checkpoint_files/model.ckpt-847000
INFO:transformers.modeling_electra:Loading TF weight discriminator_predictions/dense/bias with shape [768]
INFO:transformers.modeling_el

In [8]:
!cp $ELECTRA_BASE_DIR/electra_cs_base_discriminator.bin /gdrive/'My Drive'/question_answering/models/electra_cs/model_base/electra_cs_base_discriminator.bin

In [9]:
# API, doesn't work for Electra
# https://medium.com/analytics-vidhya/this-article-shows-how-to-convert-the-tensorflow-model-to-the-huggingface-transformers-model-4a39a09a2242

# !transformers-cli convert \
# --model_type bert \
# --tf_checkpoint $ELECTRA_BASE_DIR/checkpoint_files/model.ckpt-847000 \
# --pytorch_dump_output $ELECTRA_BASE_DIR/electra_cs_base_discriminator.bin \
# --config $ELECTRA_BASE_DIR/electra_cs_discriminator_config.json \

In [24]:
from transformers import ElectraForPreTraining, ElectraConfig, ElectraTokenizer
import torch

# https://huggingface.co/transformers/model_doc/electra.html#electraconfig
configuration = ElectraConfig.from_json_file('model_base/electra_cs_base_discriminator_config.json')
discriminator = ElectraForPreTraining.from_pretrained('model_base/electra_cs_base_discriminator.bin', config=configuration)
tokenizer = ElectraTokenizer("model_base/electra_cs_base_vocab.txt", do_lower_case=False)
discriminator.eval()

ElectraForPreTraining(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(127847, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, eleme

In [28]:
def test_electra_discriminator(fake_sentence, discriminator, tokenizer):
  fake_tokens = tokenizer.tokenize(fake_sentence)
  fake_inputs = tokenizer.encode(fake_sentence, return_tensors="pt")
  
  discriminator_outputs = discriminator(fake_inputs)
  predictions = torch.round((torch.sign(discriminator_outputs[0]) + 1) / 2)

  [print("%7s" % token, end="") for token in fake_tokens]

  [print("%7s" % prediction, end="") for prediction in predictions.tolist()]

In [29]:
fake_sentence = "Rychlá hnědá liška chyba přes líného psa"
test_electra_discriminator(fake_sentence=fake_sentence, discriminator=discriminator, tokenizer=tokenizer)

  Rychl    ##á  hnědá  liška  chyba   přes   líné   ##ho    psa    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0

In [30]:
fake_sentence = "Dnes jsem šla do chyba a koupila si zmrzlinu"
test_electra_discriminator(fake_sentence=fake_sentence, discriminator=discriminator, tokenizer=tokenizer)

   Dnes   jsem    šla     do  chyba      a  koupi   ##la     si  zmrzl  ##inu    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0